In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import os
from tqdm import tqdm

In [29]:
data_dir = '../Data'
num_classes = 6
batch_size = 32
img_size = 224
epochs = 10
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [31]:
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = full_dataset.classes

In [32]:
val_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [33]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

In [34]:
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, num_classes)
)

In [35]:
model = model.to(device)

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

In [37]:
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    print(f"\nEpoch {epoch+1}/{epochs}")
    loop = tqdm(train_loader, desc="Training", leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

        # Update tqdm description
        loop.set_postfix(loss=loss.item(), acc=100. * train_correct / train_total)

    train_acc = 100. * train_correct / train_total
    print(f"Epoch {epoch+1} Completed | Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")


Epoch 1/10


Epoch 1 Completed | Loss: 81.2664 | Train Acc: 51.98%

Epoch 2/10


Epoch 2 Completed | Loss: 55.7995 | Train Acc: 69.93%

Epoch 3/10


Epoch 3 Completed | Loss: 45.4932 | Train Acc: 75.52%

Epoch 4/10


Epoch 4 Completed | Loss: 43.4716 | Train Acc: 76.06%

Epoch 5/10


...

Epoch 6 Completed | Loss: 39.7476 | Train Acc: 77.25%

Epoch 7/10


Epoch 7 Completed | Loss: 38.1355 | Train Acc: 79.57%

Epoch 8/10


Epoch 8 Completed | Loss: 35.7415 | Train Acc: 79.48%

Epoch 9/10


Epoch 9 Completed | Loss: 34.7150 | Train Acc: 81.85%

Epoch 10/10


Epoch 10 Completed | Loss: 34.0394 | Train Acc: 81.45%


In [38]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

val_acc = correct / total * 100
print(f"\nValidation Accuracy: {val_acc:.2f}%")


Validation Accuracy: 79.21%


In [39]:
torch.save(model.state_dict(), "waste_classifier_resnet18.pth")
print("Model saved as waste_classifier_resnet18.pth")

Model saved as waste_classifier_resnet18.pth


In [40]:
print("Class indices:", dict(zip(class_names, range(num_classes))))

Class indices: {'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}
